# THE GOAL OF THIS FUNCTION IS TO FIND POTENTIAL LIFE NEST ON MARS

To do we'll need a SURFACE layer, a TEMPERATURE layer, and MINERALS layer (3 here)


/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\
/!\ For more infos consult the presentation of the study /!\
/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\

To find these potential life nest you'll have to follow some steps: 

1- Make sure that you had correctly downloaded the file "MappingMars"
2- Make sure that you've download ArcGIS
3- Open the Jupiter Notebook under the ArcGIS main file
4- Run this code and enjoy choosing your best criteria to find potential life nest on Mars :D



In [6]:
#C:/Users/yanis/Desktop/EPF 2nd/RUDN2/Mapping project/MappingMars


def Mapping_Potential_Life_Nest():
    import arcpy
    import os
    from arcpy.sa import Raster
    
    arcpy.env.workspace = input("Enter here the root of the MappingMars_term file, with / and not \ :")
    
    arcpy.env.overwriteOutput = True
    #Allow to overwrite files
    
    
    ###STEP 1 : THE MINERALS
    feldspar=arcpy.sa.Raster(arcpy.env.workspace + "/Minéraux/TES_Feldspar(without MOLA).png")#Affecté LE fichier raster à une variable
    silicates=arcpy.sa.Raster(arcpy.env.workspace + "/Minéraux/TES_Glass_Clay-SILICATES (without MOLA).png")
    plagioclase=arcpy.sa.Raster(arcpy.env.workspace + "/Minéraux/TES_Plagioclase (without MOLA).png")

    L=[feldspar,silicates,plagioclase]
    
    #The arithmetic sum
    print("Please choose a coefficient for each mineral which determined its predominance to sustain life, their sum must be 1 ")
    coef_sili=float(input("Importance of SILICATES, amount between 0 and 1 : ")) #PARAMETRE
    coef_felds=float(input("Importance of FELDSPAR, amount between 0 and 1 - your previous value : "))  #PARAMETRE
    coef_plagio=float(input("Importance of PLAGIOCLASE, amount between 0 and 1 - your previous values : "))   #PARAMETRE
    
    somme= coef_sili + coef_felds + coef_plagio 
    if  somme != 1:
        return("Please enter values where their summ add up to exactly 1")
    
    print("Processing...")

    SFP = feldspar*coef_sili + silicates*coef_felds + plagioclase*coef_plagio
    #SFP.save(arcpy.env.workspace  + "/Minéraux/SFP.tif")
    
    #Normalize the SFP layer from [0;X] --> [0;100]
    SFP_normalized = (SFP - SFP.minimum) / (SFP.maximum - SFP.minimum) * 100
    #SFP_normalized.save(arcpy.env.workspace + "/Minéraux/SFP_normalized.tif")
    
    ### On fait l'outcon pour que au delà du critère choisit on soit bon, sinon le pixel est éliminé
    criteria=0
    criteria="VALUE > " + input("From 0 to 100, put the mineral abundance you'd fixed as required for life to grow :")

    SFP_validity = arcpy.sa.Con(SFP_normalized, 1, 0,  criteria)
    #SFP_validity.save("C:/Users/yanis/Desktop/EPF 2nd/RUDN2/Mapping project/MappingMars/Results" + "SFP_validity.tif")
    print("The layer where your mineral condition is satisfied is ready. \n Starting to work for the temperature layer...")
    ###STEP 2 : THE TEMPERATURE
    #We find the 4 parts of the temperature layer
    tempNO=arcpy.sa.Raster(arcpy.env.workspace + "/Temperature/NO_Norma.tif")#Affecté LE fichier raster à une variable
    tempNE=arcpy.sa.Raster(arcpy.env.workspace + "/Temperature/NE_Norma.tif")
    tempSE=arcpy.sa.Raster(arcpy.env.workspace + "/Temperature/SE_Norma.tif")
    tempSO=arcpy.sa.Raster(arcpy.env.workspace + "/Temperature/SO_Norma.tif")
    
    #The temperature layer's part are already normalized from -130°C to +25°C
    L=[tempNO,tempNE,tempSE,tempSO]
    x=0
    for k in L:
        if arcpy.Exists(k):
            x += 1
    if x==4:
        print("The four parts of the Temperature's layer have been correctly extracted")
    else:
        print("There has been an error on the extraction of the Temperature's layer, please verify that the file isn't corrupt")
    
    print("The recorded temperature on Mars for this study were from -130°C to +25°C, select values within this range")
    criteria2 = "VALUE > " + input("Enter the MINIMUM temperature you are looking for in Celsius :") + " and VALUE < " + input("Enter the MAXIMUM temperature you are looking for in Celsius :")

    #NORTH WEST
    print("NORTH WEST temperature condition analysis ...")
    tempNO_validity= arcpy.sa.Con(tempNO,1, 0,  criteria2) #Détruit les pixels qui ne sont pas entre -10 et +50
    tempNO_validity.save(arcpy.env.workspace + "/Results/tempNO_validity.tif")
    #NOTH EAST
    print("NORTH EAST temperature condition analysis ...")
    tempNE_validity= arcpy.sa.Con(tempNE,1, 0,  criteria2) 
    tempNE_validity.save(arcpy.env.workspace + "/Results/tempNE_validity.tif")
    #SOUTH EAST
    print("SOUTH EAST temperature condition analysis ...")
    tempSE_validity= arcpy.sa.Con(tempSE,1, 0,  criteria2) 
    tempSE_validity.save(arcpy.env.workspace + "/Results/tempSE_validity.tif")
    #SOUTH WEST
    print("SOUTH WEST temperature condition analysis ...")
    tempSO_validity= arcpy.sa.Con(tempSO,1, 0,  criteria2) 
    tempSO_validity.save(arcpy.env.workspace + "/Results/tempSO_validity.tif")
    
    ###Try to merge the rasters together
    #arcpy.env.workspace = "C:/Users/yanis/Desktop/EPF 2nd/RUDN2/Mapping project/MappingMars"
    liste = arcpy.env.workspace + "/Results/tempNO_validity.tif"+";"+arcpy.env.workspace + "/Results/tempNE_validity.tif"+";"+arcpy.env.workspace + "/Results/tempSE_validity.tif"+";"+arcpy.env.workspace + "/Results/tempSO_validity.tif"
    print("Merging tiles together...")
    z=arcpy.MosaicToNewRaster_management(liste, "C:/Users/yanis/Desktop/EPF 2nd/RUDN2/Mapping project/MappingMars/Results", \
                                   "BigMergeTemp.tif", "",\
                                   "8_BIT_UNSIGNED","" , "1", "LAST","FIRST")
    UnifiedTemp_validity=arcpy.sa.Raster(arcpy.env.workspace+"/Results/BigMergeTemp.tif")
    
    #The sum of these two layers is the sum of the two validated criteria, it will lead to a layer with 0 or 1 or 2 as a value per pixel
    life = SFP_validity + UnifiedTemp_validity
    
    ###Then, we know that life nest are the pixels where the two conditions are applied simultaneously, so where the value is equal to 2
    print("Finding spot where both minerals and temperature condition are fulfiled ...")
    LIFENEST=arcpy.sa.Con(life,1, "",  "VALUE = 2")
    LIFENEST.save(arcpy.env.workspace + "/LIFENEST.tif")
    
    print("Your file is ready as 'LIFENEST' in the main MappinMars folder. \n Open it on an ArcGIS or equivalent software and superpose it on the Mars' surface to find the POTENTIA LIFE NEST ON MARS given your particular condition! \n Or directly open the map nammed 'NormalyGoodOne' on the eponyme file !")


In [7]:
Mapping_Potential_Life_Nest()

Enter here the root of the MappingMars_term file, with / and not \ :C:/Users/yanis/Desktop/EPF 2nd/RUDN2/Mapping project/MappingMars
Please choose a coefficient for each mineral which determined its predominance to sustain life, their sum must be 1 
Importance of SILICATES, amount between 0 and 1 : 0.8
Importance of FELDSPAR, amount between 0 and 1 - your previous value : 0.1
Importance of PLAGIOCLASE, amount between 0 and 1 - your previous values : 0.1
Processing...
From 0 to 100, put the mineral abundance you'd fixed as required for life to grow :40
The layer where your mineral condition is satisfied is ready. 
 Starting to work for the temperature layer...
The four parts of the Temperature's layer have been correctly extracted
The recorded temperature on Mars for this study were from -130°C to +25°C, select values within this range
Enter the MINIMUM temperature you are looking for in Celsius :-30
Enter the MAXIMUM temperature you are looking for in Celsius :50
NORTH WEST temperature